In [40]:
import random
import pandas as pd
import numpy as np

In [41]:
# inputs
num_tipos_pacientes = 10
num_recursos = 12
num_areas_pop = 70
num_hospitais = 10
size_square = 100

horizonte_planejado = 13

# valores maximos
estadia_max = 6 #round(horizonte_planejado/2)
qtd_pacientes_por_tipo_area_max = 400
qtd_recursos_por_hospital_max = 290
qtd_recursos_por_hospital_min = 250
distancia_max = 50
data_saida_pac_init_max = estadia_max - 1
qtd_pacientes_liberados_init_max = qtd_recursos_por_hospital_max/2


In [42]:
# tipo de paciente vs recursos usados

tipo_paciente = ["TP" + str(i) for i in range(1, num_tipos_pacientes + 1)]
recursos =  ["R" + str(i) for i in range(1, num_recursos + 1)]
usa_recurso = np.random.choice(['0', '1'], size=(num_tipos_pacientes*num_recursos))

tipo_paciente_df_in = np.tile(tipo_paciente, num_recursos)
tipo_paciente_df = sorted(tipo_paciente_df_in)
recursos_df = np.tile(recursos, num_tipos_pacientes)

#print(tipo_paciente)
#print(recursos)
#print(usa_recurso)

paciente_vs_recurso = pd.DataFrame({'Tipo_de_paciente': tipo_paciente_df, 'Recurso': recursos_df, 'Usa_recurso': usa_recurso})
paciente_vs_recurso['Usa_recurso'] = pd.to_numeric(paciente_vs_recurso['Usa_recurso'])
  
#display(paciente_vs_recurso.sort_values(by=['Tipo_de_paciente', 'Recurso']))

#fazer check se para cada tipo de paciente tem pelo menos 1 recurso sendo alocado
for tipo_paciente_in in paciente_vs_recurso['Tipo_de_paciente'].unique():
  filtro_tipo = paciente_vs_recurso['Tipo_de_paciente'] == tipo_paciente_in
 
  soma_recursos = paciente_vs_recurso.loc[filtro_tipo, 'Usa_recurso'].sum()
  
  if soma_recursos == 0:

      filtro_recurso_1 = paciente_vs_recurso['Recurso'] == 'R1'
      
      paciente_vs_recurso.loc[filtro_tipo & filtro_recurso_1, 'Usa_recurso'] = 1

paciente_vs_recurso.sort_values(by=['Tipo_de_paciente', 'Recurso'])

,Tipo_de_paciente,Recurso,Usa_recurso
0,TP1,R1,0
9,TP1,R10,1
10,TP1,R11,0
11,TP1,R12,0
1,TP1,R2,1
...,...,...,...
112,TP9,R5,0
113,TP9,R6,1
114,TP9,R7,0
115,TP9,R8,0


In [43]:
# tipo de paciente vs tempo de estadia
amostras_normais = np.random.normal(loc=estadia_max / 2, scale=estadia_max / 5, size=num_tipos_pacientes)
amostras_limite = np.clip(np.round(amostras_normais), 1, estadia_max)
amostras_limite = [max(elem, 0)  for elem in amostras_limite]
estadia = amostras_limite #np.random.randint(1, estadia_max, size=num_tipos_pacientes)

#print(tipo_paciente)
#print(estadia)

tipo_paciente_estadia_df = pd.DataFrame({'Tipo_de_paciente': tipo_paciente, 'Estadia': estadia})
display(tipo_paciente_estadia_df)

,Tipo_de_paciente,Estadia
0,TP1,1.0
1,TP2,1.0
2,TP3,1.0
3,TP4,4.0
4,TP5,4.0
5,TP6,1.0
6,TP7,3.0
7,TP8,4.0
8,TP9,5.0
9,TP10,2.0


In [44]:
# tipo de paciente vs área populacional vs dia de entrada vs qtd pacientes
area_pop = ["AP" + str(i) for i in range(1, num_areas_pop + 1)]
dias_planejados = [int(i) for i in range(1, horizonte_planejado + 1)] 
tipo_paciente = ["TP" + str(i) for i in range(1, num_tipos_pacientes + 1)]


qtd_pacientes_normais = np.random.normal(
    loc=qtd_pacientes_por_tipo_area_max / 2, 
    scale=qtd_pacientes_por_tipo_area_max/5, 
    size=num_tipos_pacientes * num_areas_pop * horizonte_planejado
)
qtd_pacientes_limite = np.clip(np.round(qtd_pacientes_normais), 0, qtd_pacientes_por_tipo_area_max)
qtd_pacientes_limite = [max(elem, 0) for elem in qtd_pacientes_limite]

qtd_pacientes = qtd_pacientes_limite #np.random.randint(0, qtd_pacientes_por_tipo_area_max, size=num_tipos_pacientes*num_areas_pop*horizonte_planejado)

tipo_paciente_df = np.tile(tipo_paciente, num_areas_pop*horizonte_planejado)

area_pop_df_in = np.tile(area_pop, num_tipos_pacientes*horizonte_planejado)
area_pop_df = sorted(area_pop_df_in)

dias_planejados_df_in1 = np.tile(dias_planejados, num_areas_pop)

dias_planejados_df_in2 = sorted(dias_planejados_df_in1)

dias_planejados_df = np.tile(dias_planejados_df_in2, num_tipos_pacientes)

area_pop_qtd_pac_df = pd.DataFrame({'Area_pop': area_pop_df, 'Tipo_de_paciente': tipo_paciente_df, 'Qtd_pacientes': qtd_pacientes, 'Dia': dias_planejados_df})
display(area_pop_qtd_pac_df)

,Area_pop,Tipo_de_paciente,Qtd_pacientes,Dia
0,AP1,TP1,189.0,1
1,AP1,TP2,215.0,1
2,AP1,TP3,188.0,1
3,AP1,TP4,162.0,1
4,AP1,TP5,222.0,1
...,...,...,...,...
9095,AP9,TP6,112.0,13
9096,AP9,TP7,305.0,13
9097,AP9,TP8,67.0,13
9098,AP9,TP9,279.0,13


In [45]:
# hospitais vs recursos vs quantidade de recursos
hospitais = ["H" + str(i) for i in range(1, num_hospitais + 1)]

media = (qtd_recursos_por_hospital_min + qtd_recursos_por_hospital_max) / 2
desvio_padrao = (qtd_recursos_por_hospital_max - qtd_recursos_por_hospital_min) / 6
qtd_recursos_normais = np.random.normal(
    loc=media, 
    scale=desvio_padrao, 
    size=num_recursos * num_hospitais
)
qtd_recursos_limite = np.clip(np.round(qtd_recursos_normais), qtd_recursos_por_hospital_min, qtd_recursos_por_hospital_max)
qtd_recursos_limite = [max(elem,0) for elem in qtd_recursos_limite]

qtd_recursos = qtd_recursos_limite # np.random.randint(qtd_recursos_por_hospital_min, qtd_recursos_por_hospital_max, size=num_recursos*num_hospitais)

hospitais_df = np.tile(hospitais, num_recursos)
recursos_df_in = np.tile(recursos, num_hospitais)
recursos_df = sorted(recursos_df_in)
#print(hospitais_df)
#print(qtd_recursos)

hospitais_vs_rec_qtd_df = pd.DataFrame ({'Hospital': hospitais_df, 'Recurso': recursos_df, 'Qtd_recurso': qtd_recursos})
display(hospitais_vs_rec_qtd_df.sort_values(by=['Hospital','Recurso']))

,Hospital,Recurso,Qtd_recurso
0,H1,R1,256.0
10,H1,R10,271.0
20,H1,R11,258.0
30,H1,R12,269.0
40,H1,R2,251.0
...,...,...,...
78,H9,R5,276.0
88,H9,R6,274.0
98,H9,R7,274.0
108,H9,R8,271.0


In [46]:
# Função para calcular a distância euclidiana
def calcular_distancia(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

# Função principal
def calcular_distancias(hospitais, area_pop):
    # Dicionários para armazenar as localizações dos hospitais e áreas populacionais
    localizacao_hospitais = {}
    localizacao_area_pop = {}
    
    # Atribuir localizações aleatórias no plano cartesiano (x, y entre 0 e 100)
    for hospital in hospitais:
        localizacao_hospitais[hospital] = (random.randint(0, size_square), random.randint(0, size_square))
        
    for area in area_pop:
        localizacao_area_pop[area] = (random.randint(0, size_square), random.randint(0, size_square))
    
    # Criar lista para armazenar as distâncias
    distancias = []
    
    # Calcular as distâncias euclidianas entre cada hospital e cada área populacional
    for hospital in hospitais:
        for area in area_pop:
            dist = calcular_distancia(localizacao_hospitais[hospital], localizacao_area_pop[area])
            distancias.append([hospital, area, round(dist, 2)])  # Arredonda a distância para 2 casas decimais
    
    # Criar DataFrame
    df = pd.DataFrame(distancias, columns=["Hospital", "Area_pop", "Distancia"])
    
    return df


In [47]:
# hospitais vs area pop vs distância (gerando aleatoriamente pontos num plano cartesiano)
hospitais_vs_area_pop_dist_df = calcular_distancias(hospitais, area_pop)
hospitais_vs_area_pop_dist_df

,Hospital,Area_pop,Distancia
0,H1,AP1,27.78
1,H1,AP2,60.00
2,H1,AP3,38.01
3,H1,AP4,33.54
4,H1,AP5,46.53
...,...,...,...
695,H10,AP66,61.22
696,H10,AP67,68.25
697,H10,AP68,43.05
698,H10,AP69,98.49


In [48]:
# # hospitais vs area pop vs distância
# distancia = np.random.randint(1, distancia_max, size=num_hospitais*num_areas_pop)

# hospitais_dist_df = np.tile(hospitais, num_areas_pop)
# area_pop_dist_df_in = np.tile(area_pop, num_hospitais)
# area_pop_dist_df = sorted(area_pop_dist_df_in)

# hospitais_vs_area_pop_dist_df = pd.DataFrame ({'Hospital': hospitais_dist_df, 'Area_pop': area_pop_dist_df, 'Distancia': distancia})
# display(hospitais_vs_area_pop_dist_df.sort_values(by=['Hospital','Area_pop']))

In [49]:
area_pop

['AP1',
 'AP2',
 'AP3',
 'AP4',
 'AP5',
 'AP6',
 'AP7',
 'AP8',
 'AP9',
 'AP10',
 'AP11',
 'AP12',
 'AP13',
 'AP14',
 'AP15',
 'AP16',
 'AP17',
 'AP18',
 'AP19',
 'AP20',
 'AP21',
 'AP22',
 'AP23',
 'AP24',
 'AP25',
 'AP26',
 'AP27',
 'AP28',
 'AP29',
 'AP30',
 'AP31',
 'AP32',
 'AP33',
 'AP34',
 'AP35',
 'AP36',
 'AP37',
 'AP38',
 'AP39',
 'AP40',
 'AP41',
 'AP42',
 'AP43',
 'AP44',
 'AP45',
 'AP46',
 'AP47',
 'AP48',
 'AP49',
 'AP50',
 'AP51',
 'AP52',
 'AP53',
 'AP54',
 'AP55',
 'AP56',
 'AP57',
 'AP58',
 'AP59',
 'AP60',
 'AP61',
 'AP62',
 'AP63',
 'AP64',
 'AP65',
 'AP66',
 'AP67',
 'AP68',
 'AP69',
 'AP70']

In [50]:
# hospitais vs area pop vs distância
num_areas_pop

70

In [51]:
# condição inicial: hospitais vs tipo paciente vs qtd vs data de saída
data_saida = np.random.randint(1, data_saida_pac_init_max, size=num_hospitais*num_tipos_pacientes)
qtd_pacientes_liberados_init = np.random.randint (0, qtd_pacientes_liberados_init_max, size = num_hospitais*num_tipos_pacientes)

print(hospitais)
print(tipo_paciente)
print(qtd_pacientes_liberados_init)
print(data_saida)

hospitais_lib_df_in = np.tile(hospitais, num_tipos_pacientes)
hospitais_lib_df = sorted(hospitais_lib_df_in)
tipo_paciente_lib_df = np.tile(tipo_paciente, num_hospitais)

cond_init_df = pd.DataFrame({'Hospital': hospitais_lib_df, 'Tipo_de_paciente': tipo_paciente_lib_df, 'Qtd_pacientes_liberados': qtd_pacientes_liberados_init, 'Dia': data_saida})
display(cond_init_df)

['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9', 'H10']
['TP1', 'TP2', 'TP3', 'TP4', 'TP5', 'TP6', 'TP7', 'TP8', 'TP9', 'TP10']
[ 85 114 139 109  88  84  43 119  92  72 107  21  72 121 114 130  27  57
 117 101  21  22  91  86  74 138   0 111 132  96  76  88  33  67 105  18
  84  38  73  55 113  32 119  78 108  95  92 143  58  94  82 104 118 126
  58  56 143  87   4  82  36 134  54  80  66 107  17  47 110 103 138  16
  67  23 144  75 144  95 142 128 117  53 104  46  60  57 107   9 131  18
 104 108 106 105  22 110 132  47 132  84]
[4 3 3 1 2 4 4 3 4 1 1 3 2 2 3 1 1 2 2 1 2 3 3 3 2 3 1 3 2 2 1 3 1 3 3 1 4
 2 4 1 4 2 3 3 4 3 2 1 2 1 4 2 3 4 2 1 3 2 3 4 3 1 2 4 1 3 3 1 1 3 4 1 2 2
 4 1 3 1 2 1 1 4 1 4 1 3 3 4 3 2 3 2 2 2 4 3 4 4 3 4]


,Hospital,Tipo_de_paciente,Qtd_pacientes_liberados,Dia
0,H1,TP1,85,4
1,H1,TP2,114,3
2,H1,TP3,139,3
3,H1,TP4,109,1
4,H1,TP5,88,2
...,...,...,...,...
95,H9,TP6,110,3
96,H9,TP7,132,4
97,H9,TP8,47,4
98,H9,TP9,132,3


In [52]:
paciente_vs_recurso.to_csv('../data/paciente_vs_recurso.csv', index=False)
tipo_paciente_estadia_df.to_csv('../data/tipo_paciente_estadia.csv', index=False)
area_pop_qtd_pac_df.to_csv('../data/area_pop_qtd_pacientes.csv', index=False)
hospitais_vs_rec_qtd_df.to_csv('../data/hospitais_vs_recursos_qtd.csv', index=False)
hospitais_vs_area_pop_dist_df.to_csv('../data/hospitais_vs_area_pop_dist.csv', index=False)
cond_init_df.to_csv('../data/cond_init.csv', index=False)